In [ ]:
!pip install langchain langchain-community langchain-openai
!pip install faiss-cpu
!pip install pymupdf
!pip install sentence-transformers
!pip install openai
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
naver news api
"""

import urllib.request
import urllib.parse
import json
# from dotenv import load_dotenv
import os
import time
import openai
import ast

class NaverNewsAPI:
    def __init__(self):
        # load_dotenv()
        # self.client_id = os.getenv("NAVER_API_CLIENT_ID")
        # self.client_secret = os.getenv("NAVER_API_CLIENT_SECRET")
        self.client_id = "EPCR5d1whNbimUA9ICpK"
        self.client_secret = "oAjgY5t6Pi"
        self.base_url = "https://openapi.naver.com/v1/search/news.json"

    def search(self, keyword: str, display: int = 100) -> dict:
        enc_text = urllib.parse.quote(keyword)
        url = f"{self.base_url}?query={enc_text}&display={display}"

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", self.client_id)
        request.add_header("X-Naver-Client-Secret", self.client_secret)

        try:
            with urllib.request.urlopen(request) as response:
                if response.getcode() == 200:
                    response_body = response.read().decode('utf-8')
                    print("✅ request Success")
                    return json.loads(response_body)
                else:
                    print(f"❌ Error Code: {response.getcode()}")
                    return {}
        except Exception as e:
            print(f"❌ Request failed: {e}")
            return {}

In [ ]:
# gpt 로 돌리는 경우

"""
news search
"""

# ✅ API 키 설정
openai.api_key = ""
news = NaverNewsAPI()

def gpt_prompt_action(prompt: str, max_tokens: int):
  response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=max_tokens
  )

  return response.choices[0].message.content.strip()


def summary(text: str, keyword: str):
  prompt = f"""
너는 뉴스 분석 전문가야. 다음은 뉴스 기사 전문이야.

[뉴스 기사 전문]
{text}

이 기사에서 "{keyword}"와 관련된 내용이 있다면, 관련된 내용을 요약해서 알려줘.
만약 관련 내용이 없다면 부가 설명 없이 "관련 없음" 이란 단어만 말해줘.
"""

  return gpt_prompt_action(prompt,300)


def news_filter(news_list: list, search_word: str) :
  filter_ls = [x.replace('"', '') for x in news_list if '관련 없음' not in x]
  count = len(filter_ls)
  concat_text = '\n'.join(filter_ls)
  prompt = f"""
아래 문장들을 보고 {search_word} 주제 기준으로 긍정적인지 부정적인지 알려줘.

{concat_text}

답변은 부가 설명없이 아래 list안에 json 형식을 담아서 답변해줘.
요소인 json 형식은 아래와 같아. 총 {count}개 문장이니 리스트에 요소 확실히 개수 맞춰서 대답해줘.

"""

  add_prompt = """
{
    'num' : '위에 나오는 문장의 순서',
    'sentiment' : '긍정 or 부정'
}

답변은 꼭 리스트로 해줘
  """

  return gpt_prompt_action(prompt+add_prompt, 4000)


def news_final_summary(candidate: str,search_word: str) :

    result = news.search(keyword = candidate + " " + search_word)

    check_ls = []
    for item in result.get("items", []):
        result = summary(item['title'] + item['description'], search_word)
        check_ls.append(result)

    filter_ls = [x.replace('"', '') for x in check_ls if '관련 없음' not in x]
    concat_text = '\n'.join(filter_ls)
    prompt = f"""
너는 정책 분석 전문가야.
아래 문장들을 보고 {candidate}후보의 {search_word} 주제 기준으로 핵심적인 정책을 구체적으로 요약해줘.

중요한건 {candidate} 후보의 정책이어야 해. 다른 사람의 정책 관련 문장이면 무시해.

{concat_text}

답변은 부가 설명없이 구체적인 정책 요약만 해줘.
"""

    return gpt_prompt_action(prompt, 4000)

def news_sentiment_action(search_word: str) :
    # main.py
    result = news.search(keyword = search_word)

    # print(f"result - {result}")

    # 예시 출력
    check_ls = []
    for item in result.get("items", []):
        result = summary(item['title'] + item['description'], search_word)
        check_ls.append(result)
        # 여기서 요약 태워서 김문수 후보의 청년 주거 정책에 관련된 뉴스가 맞는지? 맞으면 그 내용은 요약해서 무엇인지 내놓게 하기
        # 그다음 해당 내용들을 기준으로 감성 분석 진행 -> 100개 기준 몇개가 긍정적인지 부정적일지로 해서

    final_result = news_filter(check_ls, search_word)

    # 문자열 → 리스트로 안전하게 변환

    retries = 0
    max_retries = 3
    while retries < max_retries:
        try:
            data = ast.literal_eval(final_result)
            if isinstance(data, list):  # 리스트인지 확인
                # return data
                # 긍정/부정 개수 세기
                total = len(data)
                positive = sum(1 for item in data if item['sentiment'] == '긍정')
                negative = sum(1 for item in data if item['sentiment'] == '부정')

                # 비율 계산
                positive_ratio = round(positive / total * 100, 2)
                negative_ratio = round(negative / total * 100, 2)

                print(f"총 개수: {total}")
                print(f"긍정: {positive}개 ({positive_ratio}%)")
                print(f"부정: {negative}개 ({negative_ratio}%)")

                final_result = f"{search_word} - 긍정({positive_ratio}) / 부정({negative_ratio})"

                return final_result
            else:
                print("⚠️ 변환 성공했으나 리스트 타입이 아닙니다.")
                return None
        except (ValueError, SyntaxError) as e:
            print(f"❌ ast.literal_eval 실패 (시도 {retries + 1}/{max_retries}): {e}")
            retries += 1
            time.sleep(1)

    return None


In [ ]:
import os
import re
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQAWithSourcesChain
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ✅ 모델 초기화
llm = ChatOpenAI(temperature=0.3)

# ✅ 후보자 정보
candidates = ["이재명", "김문수", "이준석", "권영국", "송진호"]
PDF_FOLDER = "/content/drive/MyDrive/대학원/00.수업자료/00.25년도_1학기/00.빅데이터자연어처리/01.팀플/00.data/"

file_paths = {
    name: [f"{PDF_FOLDER}20250604_대한민국_{name}_선거공약서.pdf"] * 2
    for name in candidates
}

# ✅ 문서 로딩
all_documents = []
for name, paths in file_paths.items():
    for path in paths:
        loader = PyMuPDFLoader(path)
        data = loader.load()
        for d in data:
            d.metadata["candidate"] = name
            page = d.metadata.get("page", "?")
            d.metadata["source"] = f"{os.path.basename(path)}:p{page}"
        all_documents.extend(data)

# ✅ 문서 분할
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200, encoding_name='cl100k_base'
)
documents = splitter.split_documents(all_documents)

# ✅ 임베딩 및 벡터스토어 생성
embedding_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory="chroma_db")
vectorstore.persist()

# ✅ 후보별 Retriever
retrievers = {
    c: vectorstore.as_retriever(search_kwargs={"k": 6, "filter": {"candidate": c}})
    for c in candidates
}

# ✅ 프롬프트
policy_prompt = PromptTemplate(
    input_variables=["summaries", "question"],
    template="""
아래 문서는 대통령 후보의 공약이다. 질문에 대해 다음 요소를 포함하여 답변하라:
- 정책의 목적
- 구체적 수단 (시설, 제도, 법안 등)
- 실행 대상 또는 지역
- 문서상 등장한 구체적인 단어(용어)를 사용
- 반드시 한국어로 답변할 것

문맥:
{summaries}

질문:
{question}

답변:
"""
)

compare_prompt_5way = PromptTemplate(
    input_variables=["topic", "comparisons"],
    template="""
다음은 '{topic}'에 대한 대통령 후보 공약 요약이다. 다음 기준에 따라 자세히 비교하라:

1. 정책의 목적 비교
2. 구체적 수단 비교
3. 실행 대상 또는 지역 비교
4. 문서상 등장한 구체적 용어 비교

아래 형식을 유지하고 문장을 요약하지 마시오. 반드시 문단 단위로 상세히 작성하고, 후보별 차이점을 구체적으로 명시하시오.

후보별 공약:
{comparisons}

반드시 한국어로 작성하시오.
"""
)

translation_prompt = PromptTemplate(
    input_variables=["english_text"],
    template="다음 영어 텍스트를 자연스럽고 정확한 한국어로 번역하십시오:\n\n{english_text}\n\n번역:"
)

candidate_detect_prompt = PromptTemplate(
    input_variables=["question"],
    template="다음 질문에서 언급된 대통령 후보의 이름을 정확히 추출하시오. 두 명 이상일 경우 모두 출력하시오.\n\n질문: {question}\n후보 이름:"
)

# ✅ 체인 구성
qa_chains = {
    name: RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        retriever=retrievers[name],
        chain_type="stuff",
        chain_type_kwargs={"prompt": policy_prompt},
        return_source_documents=True
    ) for name in candidates
}
compare_chain_5way = LLMChain(llm=llm, prompt=compare_prompt_5way)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)
candidate_chain = LLMChain(llm=llm, prompt=candidate_detect_prompt)

# ✅ 유틸 함수
def is_empty_or_irrelevant(answer: str) -> bool:
    patterns = ["관련.*없", "언급되지 않았습니다", "포함되어 있지 않습니다", "등장하지 않습니다", "찾을 수 없습니다", "문서에.*없"]
    return not answer.strip() or any(re.search(p, answer) for p in patterns)

def is_english(text: str, threshold: float = 0.6) -> bool:
    english_chars = re.findall(r'[a-zA-Z]', text)
    total_chars = re.findall(r'\S', text)
    return bool(total_chars) and len(english_chars) / len(total_chars) >= threshold

def translate_if_needed(text: str) -> str:
    if is_english(text):
        result = translation_chain.invoke({"english_text": text})
        return result["text"] if isinstance(result, dict) else result
    return text

# ✅ 외부 뉴스 요약 fallback (news_final_summary는 외부에서 정의)
def fallback_policy_response(candidate: str, keyword: str) -> str:
    try:
        summary_result = news_final_summary(candidate, keyword)
        return summary_result or f"{candidate} 후보의 '{keyword}' 관련 정책 정보를 외부에서 찾을 수 없습니다."
    except Exception as e:
        return f"{candidate} 후보의 '{keyword}' 관련 외부 뉴스 요약 중 오류가 발생했습니다: {e}"

def format_candidate_policy(candidate: str, answer: str, is_external: bool) -> str:
    source = "📰 외부 뉴스 요약 기반:" if is_external else "📄 PDF 기반 공약:"
    return f"[{candidate} 후보]\n{source}\n{answer.strip()}"

def format_final_comparison(topic: str, comparisons: list[str]) -> str:
    return f"""✅ '{topic}'에 대한 후보별 공약 비교 분석 결과\n\n{chr(10).join(comparisons)}\n\n※ 일부 후보의 경우 PDF에 해당 정책이 없어 외부 뉴스 요약 정보가 사용되었습니다."""

# ✅ 단일 후보 질의
def run_candidate_policy_qa(input):
    result = candidate_chain.invoke({"question": input})["text"]
    target = next((c for c in candidates if c in result), None)
    if not target:
        return "후보 이름을 인식할 수 없습니다."

    try:
        answer = qa_chains[target].invoke({"question": input})["answer"]
        is_external = is_empty_or_irrelevant(answer)
        if is_external:
            answer = fallback_policy_response(target, input)
    except Exception:
        is_external = True
        answer = fallback_policy_response(target, input)

    answer = translate_if_needed(answer)
    return f"Final Answer:\n{format_candidate_policy(target, answer, is_external)}"

# ✅ 다자 비교
def run_policy_compare_all(input):
    if "," in input:
        split = [c.strip() for c in input.split(",")]
        involved = [c for c in split if c in candidates]
        keyword = next((k for k in split if k not in candidates), input)
    else:
        involved = candidates
        keyword = input.strip()

    comparisons = []
    for cand in involved:
        try:
            raw_answer = qa_chains[cand].invoke({"question": keyword})["answer"]
            is_external = is_empty_or_irrelevant(raw_answer)
            answer = fallback_policy_response(cand, keyword) if is_external else raw_answer
        except Exception:
            is_external = True
            answer = fallback_policy_response(cand, keyword)

        answer = translate_if_needed(answer)
        comparisons.append(format_candidate_policy(cand, answer, is_external))

    result = compare_chain_5way.invoke({
        "topic": keyword,
        "comparisons": "\n\n".join(comparisons)
    })["text"]
    result = translate_if_needed(result)

    return f"{format_final_comparison(keyword, comparisons)}\n\n✅ 최종 비교 분석\n{result}"

# ✅ 툴 & 에이전트 설정
react_tools = [
    Tool(name="CandidatePolicyQA", func=run_candidate_policy_qa, description="질문에서 후보를 식별하고 해당 공약을 검색함."),
    Tool(name="ComparePolicies", func=run_policy_compare_all, description="모든 후보의 공약을 비교하고 정책이 없으면 외부 함수 호출로 보완함.")
]

agent = initialize_agent(
    tools=react_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        "system_message": (
            "당신은 반드시 한국어로 사고하고 응답하는 정책 분석 도우미입니다. "
            "Thought, Action, Observation, Final Answer 형식을 사용하며, 모든 출력은 한국어여야 합니다. "
            "**Final Answer는 반드시 정책 비교의 세부 내용을 포함하여 문단 형태로 자세히 작성하십시오.**"
        )
    },
    handle_parsing_errors=True
)


### 실험

In [ ]:
def action_query(query) :
  # query = "이재명과 김문수 후보 두명의 노인 관련 정책 비교해줘"

  response = agent.invoke({"input": query})["output"]

  if is_english(response):
      translated = translation_chain.invoke({"english_text": response})
      if isinstance(translated, dict):
          print(translated["text"])
      else:
          print(translated)  # fallback: translated가 str인 경우
  else:
      print(response)

In [ ]:
query = "이재명과 김문수 후보 두명의 노인 관련 정책 비교해줘"
action_query(query)



> Entering new AgentExecutor chain...
I should compare the policies of Lee Jae-myung and Kim Moon-soo regarding the elderly.
Action: ComparePolicies
Action Input: 이재명, 김문수, 노인
Observation: Final Answer:
이재명 후보의 공약서에는 노인에 대한 구체적인 내용이 언급되지 않았습니다. 반면 김문수 후보의 공약서에는 노인을 대상으로 한 정책이 명확히 제시되었습니다. 김문수 후보는 어르신들의 건강과 복지를 향상시키기 위해 사회 서비스형 어르신 복지 일자리 확충, 어르신 데이케어센터 이용시간 확대, 요양병원 간병비 건강보험 적용, 치매 안심 국가책임제 강화, 어르신 건강심부름택시 운영 등을 구체적 수단으로 제시하였습니다. 따라서 김문수 후보의 공약은 노인을 중심으로 한 복지 정책에 대한 내용이 풍부하게 담겨 있습니다.
Thought:Based on the comparison of policies, Kim Moon-soo has more specific policies regarding the elderly compared to Lee Jae-myung.
Final Answer: Kim Moon-soo has more detailed policies for the elderly compared to Lee Jae-myung.

> Finished chain.
김문수는 이재명보다 노인들을 위한 더 구체적인 정책을 가지고 있습니다.


In [ ]:
query = "각 후보별 노인 관련 정책 비교해줘"
action_query(query)



> Entering new AgentExecutor chain...
I should compare the policies of each candidate related to the elderly.
Action: ComparePolicies
Action Input: Elderly policies
Observation: Final Answer:
이재명 후보의 노인 정책은 노인들의 삶의 질을 향상시키고 사회적 참여를 촉진하여 노인들의 복지를 증진시키는 것을 목적으로 하며, 전국적으로 노인들을 대상으로 한다. 구체적인 수단으로는 노인 복지시설 확충, 노인 건강관리 프로그램 강화, 노인 일자리 창출 등이 제시되었다. 반면 권영국 후보는 노인 정책을 통해 경제적 불평등을 해소하고 부의 격차를 축소하며, 노인들의 복지를 확대하는 것을 목적으로 하며, 부유한 계층을 대상으로 한다. 구체적인 수단으로는 상속‧증여세 90% 인상, 순자산 기준 부유세 신설, 소득세‧법인세 할증, 금융투자소득세와 가상자산세 30% 과세 등이 제시되었다.

김문수 후보는 중산층의 자산증식과 생활 수준 향상을 돕기 위한 정책을 제시하였다. 이 정책은 중산층을 대상으로 하며, 종합소득세 물가연동제 도입, 소득세 기본공제 상향 조정, 장기주식보유자나 펀드에 세제 혜택 제공 등이 구체적인 수단으로 제시되었다. 이에 비해 이준석 후보는 지방자치단체의 법인세 자치권을 확대하여 지역 경제 활성화 및 지자체 간 경쟁을 촉진하는 정책을 제시하였다. 이 정책은 지방자치단체와 기업을 대상으로 하며, 법인세 국세분의 일부를 지방세로 전환하고, 지자체별 법인세 경쟁 체계를 구축하는 것이 주요 수단으로 제시되었다.
Thought:I have compared the elderly policies of each candidate and gathered information on their specific proposals.
Final Answer: 이재명 후보 focuses on improving the quality of

In [ ]:
query = "각 후보별 수산물에 관련된 정책 비교해줘"
action_query(query)



> Entering new AgentExecutor chain...
우선 각 후보의 수산물 관련 정책을 비교해야 할 것이다.
Action: ComparePolicies
Action Input: 수산물
Observation: Final Answer:
이재명 후보의 공약은 어업협정 이행 강화와 '불법 중국어선 강력 대응'을 통해 해양 주권을 수호하는 것에 초점을 맞추고 있습니다. 이에 반해, 김문수 후보는 소상공인의 매출을 증대시키기 위해 소비를 촉진하고 전통시장과 소상공인의 경쟁력을 강화하는 방안을 제시하고 있습니다. 이준석 후보는 지방자치단체의 법인세 자치권을 확대하여 지역 경제 활성화와 지자체 간 경쟁을 촉진하고 지방 분권을 강화하는 것을 목표로 하고 있습니다. 권영국 후보는 유라시아 수출 비중을 확대하여 대미 수출 비중을 줄이고 중국 의존도를 축소하여 다각화된 공급망을 구축하는 것을 목표로 삼고 있습니다. 마지막으로, 송진호 후보는 골목경제와 서민경제를 회복하고 글로벌 금융허브로서의 위상을 강화하기 위해 전통시장 디지털화와 온라인 연계판매를 지원하고 국제금융센터 및 외국계 금융기업을 유치하는 방안을 제시하고 있습니다.
Thought:각 후보의 수산물 관련 정책은 다양하게 나타났다. 이를 종합하여 최종 답변을 내야겠다.
Action: ComparePolicies
Action Input: 수산물
Observation: Final Answer:
이재명 후보는 어업협정 이행 강화 및 '불법 중국어선 강력 대응'을 통한 해양 주권 수호를 목적으로 하고, 어업협정 강화와 불법 중국어선 대응을 구체적 수단으로 제시하며, 어업 협정 이행 대상과 해양 주권 수호 대상을 실행 대상으로 한다. 이에 대한 구체적인 용어로는 어업협정, 중국어선, 해양 주권 등이 등장한다.

김문수 후보는 소상공인의 매출을 증대시키고 전통시장과 소상공인의 경쟁력을 강화하여 서민경제를 지원하는 것을 목적으로 하며, 소비촉진을 통한 매출 증대, 소상공인 3대 패키지로의 지원, 전통시장 지원을 구체

In [ ]:
query = "김치에 관련된 정책 비교해줘"
action_query(query)



> Entering new AgentExecutor chain...
I should use the ComparePolicies function to compare policies related to kimchi.
Action: ComparePolicies
Action Input: "kimchi"✅ request Success
✅ request Success
✅ request Success
✅ request Success

Observation: ✅ 'kimchi'에 대한 후보별 공약 비교 분석 결과

[이재명 후보]
📰 외부 뉴스 요약 기반:
진천군에서 김장김치 나눔 행사, 광주 김치 축제, 김치 양배추 부족 대응, 국내외 김치 관련 이슈에 대한 대응 관련 정책을 추진해야 함.
[김문수 후보]
📰 외부 뉴스 요약 기반:
김문수 후보의 정책은 김치 생산과 나눔을 촉진하는 것이다.
[이준석 후보]
📄 PDF 기반 공약:
해당 문서에서는 '든든출발자금'이라는 정책금융상품을 통해 청년들에게 초기 자금 지원을 제공하여 기회의 평등을 실현하고자 합니다. 구체적으로는 청년기본법상 청년(만 19~34세)을 대상으로 하는 대출 상품을 제공하고, 1분기당 500만원, 최대 5,000만원 한도 내에서 연 1.7% 고정금리로 대출이 가능합니다. 이를 통해 보호종료아동, 고졸청년, 대학중퇴자 등 다양한 사각지대 청년에게 균등한 출발 기반을 제공하고자 합니다. 이 정책은 취임 후 6개월 이내에 정책 설계 및 시스템 구축을 마치고, 취임 후 1년 이내에 상품을 출시할 예정입니다. 이를 위해 연간 74조 원 규모의 예산을 통합적으로 재편성하여 재원을 마련할 것이며, 기존 청년 사업 및 대출 체계를 정비하여 예산 효율성과 형평성을 높이고자 합니다. 이를 통해 국가재정에 미치는 순손실 부담을 최소화할 것입니다.
[권영국 후보]
📰 외부 뉴스 요약 기반:
김치의 신뢰성과 안전성 확보를 위해 국가 기관이 국내 생산 농가를 지원하고 규제를 강화한다.
[송진호 후보]
📰 외부 뉴스 요약 기반:
